In [452]:
from bs4 import BeautifulSoup as BS
import re
from time import sleep
import requests
from multiprocessing.dummy import Pool as ThreadPool
from tqdm.notebook import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import pickle
import re
import json
import pandas as pd

In [163]:
driver = webdriver.Chrome()

In [164]:
driver.get('https://ranobes.com/chapters/super-god-gene/1890-glava-3-bronja-ot-suschestva-so-svjaschennoj-krovju.html')

In [165]:
page_list = []

In [167]:


page_list.append(driver.page_source)
for i in range(2):
    driver.find_element(By.XPATH, '//*[@id="next"]').click()
    sleep(0.5)
    page_list.append(driver.page_source)

In [168]:
a = driver.page_source

In [169]:
# with open('data/pages', 'wb') as f:
#     pickle.dump(page_list, f)


with open('data/pages', 'rb') as f:
    page_list_a = pickle.load(f)

In [172]:
page_list = []
page_list.append(a)
page_list.extend(page_list_a)

In [175]:
book = []


for elem in page_list[:]:
    soup = BS(elem, 'html.parser').find('div', class_='block story shortstory')
    title = soup.find('h1').find('span').previousSibling
    
    page = []
    item : BS
    for item in soup.find('h1').find_next_siblings('div')[:-2]:
        for row in item.stripped_strings:
            page.append(re.sub(r'(\xa0)', ' ',row))
    book.append((title, page))

In [187]:
# with open('data/book.json', 'w', encoding='utf-8') as f:
#     json.dump(book, f, ensure_ascii=False, indent=4)

In [465]:
with open('data/book.json', 'r', encoding='utf-8') as f:
    book = json.load(f)

In [454]:
def replace_def(pattern, repl, book: list[list[str, list[str]]]):
    new_book = []
    for title, page in book:
        new_page = []
        for row in page:
            row = re.sub(pattern, repl, row)
            new_page.append(row)
        new_book.append((title, new_page))
    return new_book

In [478]:
pattern = r'\b([Хх]ан\w?\s[Сс]ен\w{,3})\b'
book = replace_def(pattern, 'Ричи', book)
pattern = r'\b([Лл]ин\w?\sБэйфэн)'
book = replace_def(pattern, 'Маркус', book)

book = replace_def(r'(Реклама)', '', book)

In [483]:
arr = []
for i, p in book:
    arr.extend(p)

book_line = (' '.join(arr))
# pattern = r'\b([Хх]ан\w?\s[Сс]ен\w*?)\b'
pattern = r'\b([Лл]ин\w?\sБэйф\w*?)\b'
a = re.findall(pattern, book_line)
a

['Лин Бэйфэй', 'Лин Бэйфену', 'Линь Бэйфэй']

In [485]:
a = re.search(pattern, book_line)
a

<re.Match object; span=(335491, 335501), match='Лин Бэйфэй'>

In [484]:
print(book_line[7192358-100: 7192366+100])

оса. «Это должна быть она! Верно! Это Императрица Лотоса», — Ричи был почти в этом уверен. Хотя он не был уверен на сто процентов, но парень больше не смог вспомнить никого с похожим лицом. Ричи начал оглядыв


In [468]:
book[65]

('Глава 66. Король Змей с призрачными зубами',
 ['Цинь Сюань попросила Ричи тщательно подумать об Академии Черного Ястреба, потому что там он может получить хороший опыт.',
  'Ричи вышел из кабинета Цинь Сюань и думал про себя:',
  '«Цинь Сюань на самом деле добрая, но как я могу сейчас оставить мать и Янь? Я должен часто приходить домой и заботиться о них, до того как мне исполниться 20 лет и тогда мне, в любом случае, придется отправиться на службу. Но до этого момента, я должен заработать достаточно денег, чтобы они смогли хорошо жить, пока я буду служить».',
  'После того, как он вернулся домой, он позвонил Лин Бэйфэну и попросил его принести ему стрелы, содержащие Z-сталь. Он хотел отправиться в Темное Болото, как только наступит завтра. В конце концов улучшения своего физического состояния было для него в приоритете.',
  'Ян Манли была права с такими душами существ, он мог стать самым лучшим лучником. И единственное, что ему сильно не хватало — это души существа в виде стрелы.',


In [480]:
from ebooklib import epub
from random import randint

# Функция для преобразования страницы в HTML
def page_to_html(title, page):
    content_lines = page[:]
    # Формирование HTML-кода
    html = f"<h1>{title}</h1>"
    for line in content_lines:
        html += f"<p>{line}</p>"
    return html


name_book = 'super_gen'

# Создание новой книги
epub_book = epub.EpubBook()

# Установка метаданных
epub_book.set_identifier(f"id{randint(100000,1000000)}")
epub_book.set_title(name_book)
epub_book.set_language("ru")

# Автор
epub_book.add_author("None")
# epub_book.spine.append('nav')
# Обработка каждой страницы
for i, (title, page) in enumerate(book, start=1):
    # Создание главы
    chapter_content = page_to_html(title, page)
    
    chapter = epub.EpubHtml(title=title, file_name=f'chap_{i+1:04d}.xhtml', lang='ru')
    chapter.content = chapter_content
    
    epub_book.add_item(chapter)
    
    # Добавление в оглавление
    epub_book.toc.append(chapter)
    # Упаковка всех глав в книгу
    epub_book.spine.append(chapter)


# Добавление оглавления
epub_book.add_item(epub.EpubNcx())
epub_book.add_item(epub.EpubNav())

# Сохранение книги
epub.write_epub(f'{name_book}.epub', epub_book)